In [13]:
# %load src/cropper.py
from tqdm import trange
import numpy as np
from src.seam_table_service import SeamTableService
from src.energy_carving import Energy
import cv2
i_energy = Energy()


class Cropper():

    @staticmethod
    def crop_c(img, scale_c):
        r, c, _ = img.shape
        new_c = int(scale_c * c)
        for i in trange(c - new_c):
            energy_map = i_energy.calc_energy(img)
            status, img = SeamTableService.carve_column(img=img, energy_map=energy_map)

        return img

    @staticmethod
    def remove_object(img, mask):
        rotate = False
        while len(np.where(mask[:, :] > 0)[0]) > 0:

            object_height, object_width = SeamTableService.get_object_dimension(mask)
            if object_height < object_width:
                print("rotate")
                img = np.rot90(img, 1, (0, 1))
                mask = np.rot90(mask, 1, (0, 1))
                rotate = True
            else:
                print("not rotate")
            energy_map = i_energy.calc_energy(img)
            energy_map[np.where(mask[:, :] > 0)] *= -1000
            img, mask = SeamTableService.carve_column(img=img, energy_map=energy_map,
                                                                mode='object_removal', object_removal_mask=mask)
            if rotate == True:
                img = np.rot90(img, 3, (0, 1))
                mask = np.rot90(mask, 3, (0, 1))


        return img

    @staticmethod
    def crop_r(img, scale_r):
        img = np.rot90(img, 1, (0, 1))
        img = SeamTableService.crop_c(img, scale_r)
        img = np.rot90(img, 3, (0, 1))
        return img

    def save_result(out_image, filename):
        cv2.imwrite(filename, out_image.astype(np.uint8))


In [14]:
# %load src/energy_carving.py
import numpy as np
from scipy.ndimage.filters import convolve

class Energy():
    def __init__(self):
        filter_du = np.array([
            [1.0, 2.0, 1.0],
            [0.0, 0.0, 0.0],
            [-1.0, -2.0, -1.0],
        ])
        # This converts it from a 2D filter to a 3D filter, replicating the same
        # filter for each channel: R, G, B
        self.filter_du = np.stack([filter_du] * 3, axis=2)

        filter_dv = np.array([
            [1.0, 0.0, -1.0],
            [2.0, 0.0, -2.0],
            [1.0, 0.0, -1.0],
        ])
        # This converts it from a 2D filter to a 3D filter, replicating the same
        # filter for each channel: R, G, B
        self.filter_dv = np.stack([filter_dv] * 3, axis=2)
    def calc_energy(self,img):
        img = img.astype('float32')
        convolved = np.absolute(convolve(img, self.filter_du)) + np.absolute(convolve(img, self.filter_dv))
        # We sum the energies in the red, green, and blue channels
        energy_map = convolved.sum(axis=2)

        return energy_map

In [18]:
# %load src/seam_table_service.py
import numpy as np

np.bitwise_not is np.invert
class SeamTableService():

    @staticmethod
    def build_minimum_seam_table(r, c, energy_map):
        M = energy_map.copy()
        backtrack = np.zeros_like(M, dtype=np.int)

        for i in range(1, r):
            for j in range(0, c):
                # Handle the left edge of the image, to ensure we don't index -1
                if j == 0:
                    idx = np.argmin(M[i - 1, j:j + 2])
                    backtrack[i, j] = idx + j
                    min_energy = M[i - 1, idx + j]
                else:
                    idx = np.argmin(M[i - 1, j - 1:j + 2])
                    backtrack[i, j] = idx + j - 1
                    min_energy = M[i - 1, idx + j - 1]

                M[i, j] += min_energy

        return M, backtrack

    @staticmethod
    def find_seam(cumulative_map,backtrack,r, c):
        mask = np.ones((r, c), dtype=np.bool)
        j = np.argmin(cumulative_map[-1])
        for i in reversed(range(r)):
                # Mark the pixels for deletion
            mask[i, j] = False
            j = backtrack[i, j]
        return mask

    @staticmethod
    def carve_column(img, energy_map, mode=None, object_removal_mask=None):
        # r, c = energy_map.shape
        # print('energy_map.shape {} {}'.format(r, c))
        # r, c = object_removal_mask.shape
        # print('object_removal_mask.shape {} {}'.format(r, c))
        r, c, _ = img.shape
        # print('img.shape {} {}'.format(r, c))
        M,backtrack = SeamTableService.build_minimum_seam_table(r, c, energy_map)

        mask = SeamTableService.find_seam(M,backtrack, r, c)

        mask3D = np.stack([mask] * 3, axis=2)
        img = img[mask3D].reshape((r, c - 1, 3))

        if mode == 'object_removal':
            object_removal_mask = object_removal_mask[mask].reshape((r, c - 1))
            return img, object_removal_mask

        return img


    def get_object_dimension(mask):
        rows, cols = np.where(mask > 0)
        height = np.amax(rows) - np.amin(rows) + 1
        width = np.amax(cols) - np.amin(cols) + 1
        return height, width   


In [19]:
import cv2
import os
import numpy as np

drawing = False # true if mouse is pressed
mode = True # if True, draw rectangle. Press 'm' to toggle to curve
ix,iy = -1,-1
base_dir =os.path.abspath('')
# mouse callback function
def draw_circle(event,x,y,flags,param):
    global ix,iy,drawing,mode
    color = (255,255,255)
    if event == cv2.EVENT_LBUTTONDOWN:
        drawing = True
        ix,iy = x,y

    elif event == cv2.EVENT_MOUSEMOVE:
        if drawing == True:
            if mode == True:
                cv2.rectangle(img,(ix,iy),(x,y),color,-1)
            else:
                cv2.circle(img,(x,y),3,color,-1)
                cv2.circle(blackAndWhite,(x,y),3,color,-1)


    elif event == cv2.EVENT_LBUTTONUP:
        drawing = False
        if mode == True:
            cv2.rectangle(img,(ix,iy),(x,y),color,-1)
        else:
            cv2.circle(img,(x,y),3,color,-1)
            cv2.circle(blackAndWhite,(x,y),3,color,-1)

            
            
in_filename = os.path.join(base_dir, 'img\\barvaz.jpg')
img = cv2.imread(in_filename)
r, c = img.shape[: 2]
blackAndWhite  = np.zeros((r, c, 3))

cv2.namedWindow('image')
cv2.setMouseCallback('image',draw_circle)

while(1):
    cv2.imshow('image',img)
    k = cv2.waitKey(1) & 0xFF
    if k == ord('m'):
        mode = not mode
    elif k == 27:
        cv2.imwrite("img\\mask.jpg", blackAndWhite)
        break

cv2.destroyAllWindows()

In [16]:
import os
import cv2
import numpy as np

base_dir =os.path.abspath('')
mask = os.path.join(base_dir, 'img\\mask.jpg')
in_filename = os.path.join(base_dir, 'img\\barvaz.jpg')
out_filename = os.path.join(base_dir, 'img\\out.jpg')
img = cv2.imread(in_filename)
objectRemove =  cv2.imread(mask,0).astype(np.float64)
saveImage = Cropper.remove_object(img, objectRemove)
Cropper.save_result(saveImage,out_filename)


not rotate
energy_map.shape 438 438
object_removal_mask.shape 438 438
img.shape 438 438
not rotate
energy_map.shape 438 437
object_removal_mask.shape 438 437
img.shape 438 437
not rotate
energy_map.shape 438 436
object_removal_mask.shape 438 436
img.shape 438 436
not rotate
energy_map.shape 438 435
object_removal_mask.shape 438 435
img.shape 438 435
not rotate
energy_map.shape 438 434
object_removal_mask.shape 438 434
img.shape 438 434
not rotate
energy_map.shape 438 433
object_removal_mask.shape 438 433
img.shape 438 433
not rotate
energy_map.shape 438 432
object_removal_mask.shape 438 432
img.shape 438 432
not rotate
energy_map.shape 438 431
object_removal_mask.shape 438 431
img.shape 438 431
not rotate
energy_map.shape 438 430
object_removal_mask.shape 438 430
img.shape 438 430
not rotate
energy_map.shape 438 429
object_removal_mask.shape 438 429
img.shape 438 429
not rotate
energy_map.shape 438 428
object_removal_mask.shape 438 428
img.shape 438 428
not rotate
energy_map.shape 438 